# Material Parameter Optimization

In the following example, optimized values of Young's modulus, yield strength, and linear hardening parameters for a linear hardening Von Mises material model are determined using data from a standard uniaxial tension test. The data represents the response of Aluminum, alloy unkown.

The strategy is to read read data from an excel file and use the axial strain to drive the simulation. Optimization is performed by minimizing the area between the stress-strain curve calculated and the measured stress-strain curve.

<a name='contents'></a>
## Contents


In [1]:
from bokeh.io import output_notebook
from bokeh.plotting import *
import sys
import os
from matmodlab2 import *
from numpy import *
import numpy as np
from matmodlab2.optimize import Optimizer, OptimizeVariable
from matmodlab2.optimize.numerix import calculate_bounded_area, interp_rms_error
from plotting_helpers import create_figure
from pandas import read_excel
output_notebook()

Setting up the Matmodlab notebook environment


Loading BokehJS ...

In [2]:
# get the experimental stress and strain
filename = os.path.join(_dh[0], 'aldat.xls')
assert os.path.isfile(filename)
df = read_excel(filename, sheetname='MML')

## Create data to use as steps

In [3]:
num_step = 30
# Interpolate the steps
fun = lambda t: np.interp(t, df['TIME'], df['E.XX'])
timespace = np.linspace(df['TIME'].iloc[0], df['TIME'].iloc[-1], num_step)
components = []
for i in range(num_step):
    time = timespace[i]
    components.append([fun(time), 0., 0.])

In [4]:
def runjob(jobid, names, x):
    """Run a specific MaterialPointSimulator job for parameters in x"""
    xp = dict(zip(names, x))
    NU = 0.32  # poisson's ratio for aluminum
    parameters = {'K': xp['E']/3.0/(1.0-2.0*NU), 
                  'G': xp['E']/2.0/(1.0+NU),
                  'Y0': xp['Y0'], 
                  'H': xp['H'], 
                  'BETA': 0.0}
    mps = MaterialPointSimulator(jobid)
    mps.material = VonMisesMaterial(**parameters)
    for (i, item) in enumerate(components[1:], start=1):
        mps.run_step('ESS', item, increment=timespace[i])
    return mps

In [5]:
def func(x, names, evald, job, *args):
    """The objective function"""
    
    try:
        mps = runjob(job, names, x)
        failed = not mps.ran
    except:
        failed = True
    if failed:
        return 1.0e9
    #error = calculate_bounded_area(df['E.XX'], df['S.XX'], 
    #                               mps.df['E.XX'], mps.df['S.XX'])
    error = interp_rms_error(df['E.XX'], df['S.XX'], 
                                   mps.df['E.XX'], mps.df['S.XX'])
    return error

In [6]:
# set the variables to be optimized
E = OptimizeVariable('E',  2.0e6)
Y0= OptimizeVariable('Y0', 0.5e5)
H = OptimizeVariable('H',  1.0e5)
xinit = [E, Y0, H]

# the optimzer object
optimizer = Optimizer('optimize', func, xinit, method='simplex',
                      maxiter=200, tolerance=1.e-2)

# run the job
optimizer.run()

Running job 75
Done


In [7]:
print(dict(zip(optimizer.names, optimizer.xopt)))
mps = runjob('Optimized', optimizer.names, optimizer.xopt)
plot = create_figure(bokeh=True, x_axis_label='E.XX', y_axis_label='S.XX')
plot.scatter(df['E.XX'], df['S.XX'], marker='x', color='red', legend='Experimental data')
plot.line(mps.df['E.XX'], mps.df['S.XX'], legend='Simulation', color='black', line_width=3)
show(plot)

{'E': 2190878.6393605373, 'Y0': 39355.851432065654, 'H': 99568.020305288082}
